<a href="https://colab.research.google.com/github/humilitykind/Project_Optiwing/blob/main/Another_copy_of_mnist_cgan.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 3.5 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/TensorTeen/Optiwing-Cp-Plots.git

Cloning into 'Optiwing-Cp-Plots'...
remote: Enumerating objects: 3183, done.
remote: Counting objects: 100% (3183/3183), done.
remote: Compressing objects: 100% (3179/3179), done.
remote: Total 3183 (delta 3), reused 3179 (delta 2), pack-reused 0
Receiving objects: 100% (3183/3183), 4.51 MiB | 4.10 MiB/s, done.
Resolving deltas: 100% (3/3), done.


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder, MNIST
from torchvision import transforms
from torch import autograd
from torch.autograd import Variable
from torchvision.utils import make_grid
from tensorboardX import SummaryWriter
import os
import torch.optim as optim
import matplotlib.pyplot as plt
import csv
import matplotlib.ticker
import pandas as pd
from torch.utils.data import Dataset, DataLoader


In [ ]:
transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5])
])

In [ ]:
x = torch.Tensor([[1,2,3]])
x.shape

torch.Size([1, 3])

In [ ]:
class Discriminator(nn.Module):
  def  __init__(self):
        super().__init__()


        self.archtitecture=nn.Sequential(
                                         self._block(2,128,4,2),
                                         nn.LeakyReLU(0.2,inplace=True),
                                         self._block(128,128,2,2),
                                         nn.LeakyReLU(0.2,inplace=True))
        self.dense =nn.Sequential(nn.Linear(128*40,2560),nn.Linear(2560,640),nn.Linear(640,64),nn.Linear(64,1))
  def _block(self, in_channels, out_channels, kernel_size, stride, padding=0):
        return nn.Sequential(
            nn.Conv1d(
                in_channels,
                out_channels,
                kernel_size,
                stride,
                padding,
                bias=False,)
            )
  def forward(self,cp_in,y_in):
    cp_in,y_in = cp_in.reshape(len(cp_in),1,160),y_in.reshape(len(y_in),1,160)
    x = torch.concat([cp_in,y_in],dim=1)
    print(x.shape)
    x=self.archtitecture(x) #changing to 128 channels
    x=nn.Flatten(x)
    m=nn.Dropout(0.4)
    x=m(x)
    x=self.dense(x)
    return x


In [ ]:
import torch
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # Embedding layer

        # Linear layer for label
        self.label_linear=nn.Linear(160*1,40*1)
        # reshape for label
        self.label_reshape = torch.Tensor.reshape


        self.latent_linear = nn.Linear(160*1,40*1*128)

        self.leaky_relu = nn.LeakyReLU(0.2)


        self.latent_reshape = torch.Tensor.reshape


        self.concat = torch.cat

        self.deconv1 = nn.ConvTranspose1d(129, 128, kernel_size=4, stride=2, padding=1)
        self.deconv2 = nn.ConvTranspose1d(128, 128, kernel_size=4, stride=2, padding=1)

        self.output = nn.Conv1d(128, 1, kernel_size=8, padding=3)

    def forward(self, input_label, input_latent_vector):

        label_linear = self.label_linear(input_label,40*1)
        label_reshaped = self.label_reshape(label_linear,(1,40,1))

        latent_linear = self.latent_linear(input_latent_vector)
        latent_relu = self.leaky_relu(latent_linear,40*1*128)
        latent_reshaped = self.latent_reshape(latent_relu,(128, 40, 1))

        combined = self.concat([latent_reshaped, label_reshaped],dim=1)

        deconv1 = self.deconv1(combined)
        deconv1_relu = self.leaky_relu(deconv1)

        deconv2 = self.deconv2(deconv1_relu)
        deconv2_relu = self.leaky_relu(deconv2)

        output = self.output(deconv2_relu)

        return output

In [ ]:
generator = Generator()
discriminator = Discriminator()

In [ ]:
criterion = nn.MSELoss()
d_optimizer = torch.optim.Adam(discriminator.parameters(), lr=3e-4)
g_optimizer = torch.optim.Adam(generator.parameters(), lr=3e-4)

In [ ]:
writer = SummaryWriter()

In [ ]:
def generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion,labels):
    g_optimizer.zero_grad()
    z = Variable(torch.randn(batch_size, 160))
    fake_labels = Variable(labels)
    fake_images = generator(z, fake_labels)
    validity = discriminator(fake_images, fake_labels)
    g_loss = criterion(validity, Variable(torch.ones(batch_size)) )
    g_loss.backward()
    g_optimizer.step()
    return g_loss.item()

In [ ]:
def discriminator_train_step(batch_size, discriminator, generator, d_optimizer, criterion, real_images, labels):
    d_optimizer.zero_grad()

    # train with real images
    real_validity = discriminator(real_images, labels)
    real_loss = criterion(real_validity, Variable(torch.ones(batch_size)) )

    # train with fake images
    z = Variable(torch.randn(batch_size, 160))
    fake_labels = Variable(labels)
    fake_images = generator(z, fake_labels)
    fake_validity = discriminator(fake_images, fake_labels)
    fake_loss = criterion(fake_validity, Variable(torch.zeros(batch_size)) )

    d_loss = real_loss + fake_loss
    d_loss.backward()
    d_optimizer.step()
    return d_loss.item()

#Loading the Dataset

In [ ]:
#Getting the list of training Files

path = "/content/Optiwing-Cp-Plots"
os.chdir(path)
_dir = os.fsencode(path)
l = np.array([x.decode('utf-8') for x in os.listdir(_dir)])

In [ ]:
#Deserializing the data into arrays

# contains all the cp values
X = []
# Contains all the x,y values
Y = []
for i in l:
  try:
    with open(i) as f:
        try :
            next(f)
            next(f)
            next(f)
        except:
            continue
        x = []
        y = []
        cp = []
        for j in f :
            try :
                tmp = [float(j[:10]), float(j[10:19]),float(j[19:28])]
                x.append(tmp[0])
                y.append(tmp[1])
                cp.append(tmp[2])
            except :
                break
        else:
            if len(cp) == 160 :
                X.append(cp)
                Y.append([x,y])
  except IsADirectoryError:
    continue

In [ ]:
#Converting the arrays into tensors for futher processing
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(Y)
y = y.flatten(start_dim=1)

In [ ]:
#splitting the data into training and testing set

train_size = int(0.8 * len(X))
test_size = len(X) - train_size
X_train, X_test = X[:train_size],X[train_size:]
y_train, y_test = y[:train_size],y[train_size:]

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, df, training=True):
        self.X,self.y = df
        self.training=training

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        cond = self.X[idx].reshape(160,1)
        if self.training:
            label = self.y[idx][160:]
            return cond,label
        return cond


In [ ]:
train_dataset = CustomDataset((X_train,y_train), training=True)
test_dataset = CustomDataset((X_train,y_train), training=True)

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

##init weights

In [ ]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [ ]:
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

Discriminator(
  (archtitecture): Sequential(
    (0): Sequential(
      (0): Conv1d(2, 128, kernel_size=(4,), stride=(2,), bias=False)
    )
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Sequential(
      (0): Conv1d(128, 128, kernel_size=(2,), stride=(2,), bias=False)
    )
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (dense): Sequential(
    (0): Linear(in_features=5120, out_features=2560, bias=True)
    (1): Linear(in_features=2560, out_features=640, bias=True)
    (2): Linear(in_features=640, out_features=64, bias=True)
    (3): Linear(in_features=64, out_features=1, bias=True)
  )
)

In [ ]:
num_epochs = 50
n_critic = 5
display_step = 50
for epoch in range(num_epochs):
    print('Starting epoch {}...'.format(epoch), end=' ')
    for i, (images, labels) in enumerate(train_dataloader):

        step = epoch * len(train_dataloader) + i + 1
        real_images = Variable(images)
        labels = Variable(labels)
        generator.train()

        d_loss = 0
        for _ in range(n_critic):
            d_loss = discriminator_train_step(len(real_images), discriminator,
                                              generator, d_optimizer, criterion,
                                              real_images, labels)


        g_loss = generator_train_step(batch_size, discriminator, generator, g_optimizer, criterion)

        writer.add_scalars('scalars', {'g_loss': g_loss, 'd_loss': (d_loss / n_critic)}, step)

        if step % display_step == 0:
            generator.eval()
            z = Variable(torch.randn(9, 100))
            labels = Variable(torch.LongTensor(np.arange(9)))
            sample_images = generator(z, labels).unsqueeze(1)
            grid = make_grid(sample_images, nrow=3, normalize=True)
            writer.add_image('sample_image', grid, step)
    print('Done!')

Starting epoch 0... torch.Size([1, 2, 160])


TypeError: ignored

In [ ]:
torch.save(generator.state_dict(), 'generator_state.pt')

In [ ]:
z = Variable(torch.randn(100, 100))
labels = torch.LongTensor([i for i in range(10) for _ in range(10)])

In [ ]:
images = generator(z, labels).unsqueeze(1)

In [ ]:
grid = make_grid(images, nrow=10, normalize=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(grid.permute(1, 2, 0).data, cmap='binary')
ax.axis('off')

In [ ]:
def generate_digit(generator, digit):
    z = Variable(torch.randn(1, 100))
    label = torch.LongTensor([digit])
    img = generator(z, label).data.cpu()
    img = 0.5 * img + 0.5
    return transforms.ToPILImage()(img)

In [ ]:
generate_digit(generator, 4)